## Solve heat conduction on a square with a uniform area specific heat source

![cant.png](figures/heat_square.png)

In [14]:
using JuAFEM
using UnicodePlots

In [2]:
grid = JuAFEM.generate_grid(Quadrilateral, (20,20))

JuAFEM.Grid{2,4,Float64,4} with 400 Quadrilateral cells and 441 nodes


In [3]:
dim = 2
ip = Lagrange{dim, RefCube, 1}()
qr = QuadratureRule{dim, RefCube}(2)
cellvalues = CellScalarValues(qr, ip);

In [4]:
dh = DofHandler(grid)
push!(dh, :T, 1) # Add a temperature field
close!(dh)

DofHandler
  Fields:
    :T interpolation: JuAFEM.Lagrange{2,JuAFEM.RefCube,1}(), dim: 1
  Dofs per cell: 4
  Total dofs: 441

In [5]:
dbcs = ConstraintHandler(dh)
dbc = Dirichlet(:T, union(getfaceset(grid, "left"), getfaceset(grid, "right"), getfaceset(grid, "top"), getfaceset(grid, "bottom")), (x,t)->0)
add!(dbcs, dbc)
close!(dbcs)
update!(dbcs, 0.0)

In [6]:
@time K = JuAFEM.create_sparsity_pattern(dh);
fill!(K.nzval, 1.0);
spy(K)

  0.151309 seconds (61.53 k allocations: 3.535 MiB)


                    Sparsity Pattern
       ┌──────────────────────────────────────────┐    
     1 │⠻⣦⡀⠀⢷⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ > 0
       │⠀⠈⠻⣦⠈⢷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
       │⠙⠳⢦⣄⠻⣦⠻⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠻⣦⠻⣦⠻⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠻⣦⠻⣦⠻⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠻⢦⠻⣦⠻⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣦⠻⣦⠻⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣦⠻⣦⠻⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣦⠻⣦⠻⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣦⠻⣦⠻⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣦⠻⣦⠻⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⣦⠻⣦⠻⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣦⠻⣦⠻⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣦⠻⣦⠻⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣦⠻⣦⠻⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣦⠻⣦⠳⣦⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀

In [ ]:
function doassemble{dim}(cellvalues::CellScalarValues{dim}, K::SparseMatrixCSC, dh::DofHandler)
    b = 1.0
    f = zeros(ndofs(dh))
    assembler = start_assemble(K, f)
    
    n_basefuncs = getnbasefunctions(cellvalues)
    global_dofs = zeros(Int, ndofs_per_cell(dh))

    fe = zeros(n_basefuncs) # Local force vector
    Ke = zeros(n_basefuncs, n_basefuncs) # Local stiffness mastrix

    @inbounds for (cellcount, cell) in enumerate(CellIterator(dh))
        fill!(Ke, 0)
        fill!(fe, 0)
        
        reinit!(cellvalues, cell)
        for q_point in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q_point)
            for i in 1:n_basefuncs
                δT = shape_value(cellvalues, q_point, i)
                ∇δT = shape_gradient(cellvalues, q_point, i)
                fe[i] += (δT * b) * dΩ
                for j in 1:n_basefuncs
                    ∇T = shape_gradient(cellvalues, q_point, j)
                    Ke[i, j] += (∇δT ⋅ ∇T) * dΩ
                end
            end
        end
        
        celldofs!(global_dofs, cell)
        assemble!(assembler, global_dofs, fe, Ke)
    end
    return K, f
end;

In [8]:
@time K, f = doassemble(cellvalues, K, dh);

  0.499679 seconds (192.19 k allocations: 9.773 MiB)


In [ ]:
if get(ENV, "JUAFEM_TESTING", false) == "true"
    K_strat1, K_strat2 = copy(K), copy(K)
    @time apply!(K_strat1, f, dbcs; strategy = JuAFEM.APPLY_INPLACE)
    @time apply!(K_strat2, f, dbcs; strategy = JuAFEM.APPLY_TRANSPOSE)
    Test.@test K_strat1 == K_strat2
end

In [ ]:
@time apply!(K, f, dbcs)

  0.310779 seconds (143.19 k allocations: 7.183 MiB, 2.72% gc time)


In [11]:
@time T = K \ f;

  0.601325 seconds (257.06 k allocations: 13.975 MiB)


In [12]:
vtkfile = vtk_grid("heat", dh)
vtk_point_data(vtkfile, dh, T)
vtk_save(vtkfile);

norm(T)

3.307743912641305

In [13]:
Test.@test norm(T) ≈ 3.307743912641305
Test.@test maximum(T) ≈ 0.29526786377073544
println("Heat square successful")

Heat square successful
